In [1]:
%pip install langchain langchain_openai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existin

In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [3]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.json import SimpleJsonOutputParser

In [4]:
chat = ChatOpenAI(model="gpt-3.5-turbo-1106",
                  model_kwargs={'response_format': {"type": "json_object"}})

In [5]:
chat_prompt = ChatPromptTemplate.from_messages(
    ("system", """I want you to extract the person name, age and a description from the following text.
    Here is the JSON schema:
    "name": string
    "age": int
    "description": string
    {message_to_extract}
    ---
    If there are multiple people, then put them in a 'persons' key, which is a list of the above schema.
    """)
)

In [6]:
chain = (
    chat_prompt
    | chat
    | SimpleJsonOutputParser()
)

In [7]:
multiple_results = chain.invoke({
    "message_to_extract": '''James is 30 years old and he is a software engineer, he likes to play tennis and he lives in London.
    John is 35 years old and he is a data engineer and he lives in New York, He likes to play football.
    Mark is 40 years old is a Java Developer and he lives in London. He doesn't have many hobbies.
    '''
})

print(multiple_results)

{'persons': [{'name': 'James', 'age': 30, 'description': 'James is a software engineer, he likes to play tennis and he lives in London.'}, {'name': 'John', 'age': 35, 'description': 'John is a data engineer and he lives in New York, He likes to play football.'}, {'name': 'Mark', 'age': 40, 'description': "Mark is a Java Developer and he lives in London. He doesn't have many hobbies."}]}


------

## Parallel Function Calling with Pydantic

In [8]:
from pydantic.v1 import BaseModel
from typing import List
from langchain.chains.openai_tools import create_extraction_chain_pydantic

In [9]:
# Make sure to use a recent model that supports tools
model = ChatOpenAI(model="gpt-3.5-turbo-1106")

In [10]:
class Person(BaseModel):
    """A person object that we want to extract from the text"""
    name: str
    age: int

# Previous we had to write this:
class Persons(BaseModel):
    persons: List[Person]

In [11]:
chain = create_extraction_chain_pydantic(Person, model)

<ipython-input-11-16f87776c86a>:1: LangChainDeprecationWarning: LangChain has introduced a method called `with_structured_output` thatis available on ChatModels capable of tool calling.You can read more about the method here: <https://python.langchain.com/docs/modules/model_io/chat/structured_output/>. Please follow our extraction use case documentation for more guidelineson how to do information extraction with LLMs.<https://python.langchain.com/docs/use_cases/extraction/>. with_structured_output does not currently support a list of pydantic schemas. If this is a blocker or if you notice other issues, please provide feedback here:<https://github.com/langchain-ai/langchain/discussions/18154>
  chain = create_extraction_chain_pydantic(Person, model)


In [12]:
chain.invoke({"input": "James is 30 and Sarah is 26 years old."})

[Person(name='James', age=30), Person(name='Sarah', age=26)]